Load the data for super resolution problem

In [1]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("saputrahas/dataset-image-super-resolution")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/saputrahas/dataset-image-super-resolution/versions/1


What I will implement:
SRCNN https://arxiv.org/pdf/1501.00092
CoT-MISR https://arxiv.org/pdf/2303.06548
ConvNext https://huggingface.co/learn/computer-vision-course/unit2/cnns/convnext
Maybe a GAN to check the methods???? - yes, SRGAN ;-)


omg what if I tried to make low res ai generated fakes try to be real in super resolution????????


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.initializers import RandomNormal
from keras.layers import Conv2D, Conv2DTranspose, BatchNormalization, Dense, Flatten, Activation, LeakyReLU, ZeroPadding2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os
import cv2

# Prepare the dataset
first, I will divide the dataset into training and validation sets

In [3]:
from google.colab.patches import cv2_imshow

In [4]:
BASE_PATH = 'saputrahas/dataset-image-super-resolution/finished/'
TRAIN_PATH = 'finished/train/dataraw/hires/'
TEST_PATH = 'finished/valid/dataraw/hires/'
train_images = []
test_images = []

BATCH_SIZE = 64
W, H = (256, 256)

In [5]:
def get_random_crop(image, size=W):
    return tf.image.random_crop(image, [size, size, 3])

In [6]:
def get_LR(image, size=W//8):
    # image = image.numpy()
    img =  cv2.resize(image, (size, size), interpolation=cv2.INTER_CUBIC)
    return cv2.resize(img, (size*8, size*8), interpolation=cv2.INTER_CUBIC)

In [7]:
import os
import cv2

for idx, image_filename in enumerate(os.listdir(os.path.join(path, TRAIN_PATH))):
    # Construct the full image path using os.path.join
    image_path = os.path.join(path, TRAIN_PATH, image_filename)

    # Load the image using the corrected path
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    train_images.append(get_random_crop(image))

In [8]:
for idx, image_filename in enumerate(os.listdir(os.path.join(path, TEST_PATH))):
    # Construct the full image path using os.path.join
    image_path = os.path.join(path, TEST_PATH, image_filename)

    # Load the image using the corrected path
    image = cv2.cvtColor(cv2.imread(image_path), cv2.COLOR_BGR2RGB)

    test_images.append(get_random_crop(image))

In [9]:
train_images = np.array(train_images)
test_images = np.array(test_images)

In [10]:
train_images_LR = np.array([get_LR(image) for image in train_images])
test_images_LR = np.array([get_LR(image) for image in test_images])

In [11]:
[image/255 for image in train_images]
[image/255 for image in test_images]
[image/255 for image in train_images_LR]
[image/255 for image in test_images_LR]

[array([[[0.65098039, 0.64313725, 0.62352941],
         [0.65098039, 0.64313725, 0.62352941],
         [0.65098039, 0.64313725, 0.62352941],
         ...,
         [0.38431373, 0.38039216, 0.32941176],
         [0.38431373, 0.38039216, 0.32941176],
         [0.38431373, 0.38039216, 0.32941176]],
 
        [[0.65098039, 0.64705882, 0.62352941],
         [0.65098039, 0.64705882, 0.62352941],
         [0.65098039, 0.64705882, 0.62352941],
         ...,
         [0.38431373, 0.38039216, 0.33333333],
         [0.38431373, 0.38039216, 0.33333333],
         [0.38431373, 0.38039216, 0.33333333]],
 
        [[0.65098039, 0.64313725, 0.61960784],
         [0.65098039, 0.64313725, 0.61960784],
         [0.65098039, 0.64313725, 0.62352941],
         ...,
         [0.38431373, 0.37647059, 0.32941176],
         [0.38039216, 0.37647059, 0.32941176],
         [0.38039216, 0.37647059, 0.32941176]],
 
        ...,
 
        [[0.30196078, 0.31764706, 0.38431373],
         [0.30196078, 0.31764706, 0.38431

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#SRCNN
I build the SRCNN proposed in this paper: https://arxiv.org/pdf/1501.00092

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

def build_srcnn(f1=9, f2=1, f3=5, n1=64, n2=32, c=3):

    input_img = layers.Input(shape=(256, 256, c))

    conv1 = layers.Conv2D(n1, (f1, f1), padding='same', activation='relu')(input_img)

    conv2 = layers.Conv2D(n2, (f2, f2), padding='same', activation='relu')(conv1)

    conv3 = layers.Conv2D(c, (f3, f3), padding='same')(conv2)

    model = models.Model(input_img, conv3)

    return model


def compile_srcnn(model, learning_rate=1e-4):

    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])


In [16]:
if __name__ == "__main__":

    srcnn_model = build_srcnn(f1=9, f2=1, f3=5, n1=64, n2=32, c=3)
    compile_srcnn(srcnn_model, learning_rate=1e-4)
    srcnn_model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 256, 256, 64)        │          15,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 256, 256, 32)        │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 256, 256, 3)         │           2,403 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,099 (78.51 KB)

 Trainable params: 20,099 (78.51 KB)

 Non-trainable params: 0 (0.00 B)

In [17]:
srcnn_model.save('/content/drive/MyDrive/my_srcnn_model.keras')

In [13]:
from tensorflow.keras.models import load_model

In [19]:
s_model = load_model('/content/drive/MyDrive/my_srcnn_model.keras')
s_model.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 14 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 256, 256, 64)        │          15,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 256, 256, 32)        │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 256, 256, 3)         │           2,403 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 60,299 (235.55 KB)

 Trainable params: 20,099 (78.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,200 (157.04 KB)

In [14]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [21]:
checkpoint_callback = ModelCheckpoint(
    filepath='/content/drive/MyDrive/my_srcnn_model_epoch_{epoch:02d}.keras',
    save_best_only=False,
    verbose=1
)

In [22]:
srcnn_model.fit(train_images_LR[:1000], train_images[:1000], batch_size=BATCH_SIZE, epochs=10, callbacks= [checkpoint_callback])

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - loss: 15009.6514 - mse: 15009.6504 
Epoch 1: saving model to /content/drive/MyDrive/my_srcnn_model_epoch_01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 231s 14s/step - loss: 14761.6973 - mse: 14761.6953
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 15s/step - loss: 2334.8005 - mse: 2334.8005 
Epoch 2: saving model to /content/drive/MyDrive/my_srcnn_model_epoch_02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 268s 15s/step - loss: 2306.0369 - mse: 2306.0369
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - loss: 1183.9277 - mse: 1183.9276 
Epoch 3: saving model to /content/drive/MyDrive/my_srcnn_model_epoch_03.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 231s 14s/step - loss: 1176.8740 - mse: 1176.8740
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 14s/step - loss: 920.2108 - mse: 920.2108 
Epoch 4: saving model to /content/drive/MyDrive/my_srcnn_model_epoch_04.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 263s 14s/step - loss: 919.4556 - mse: 919.4556
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s

In [17]:
srcnn_model_2 = load_model('/content/drive/MyDrive/my_srcnn_model_epoch_10.keras')
srcnn_model_2.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)             │ (None, 256, 256, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d (Conv2D)                      │ (None, 256, 256, 64)        │          15,616 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_1 (Conv2D)                    │ (None, 256, 256, 32)        │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_2 (Conv2D)                    │ (None, 256, 256, 3)         │           2,403 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 60,299 (235.55 KB)

 Trainable params: 20,099 (78.51 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 40,200 (157.04 KB)

In [19]:
checkpoint_callback_2 = ModelCheckpoint(
    filepath='/content/drive/MyDrive/my_srcnn_2_model_epoch_{epoch:02d}.keras',
    save_best_only=False,
    verbose=1
)

In [21]:
srcnn_model_2.fit(train_images_LR[:1000], train_images[:1000], batch_size=BATCH_SIZE, epochs=10, callbacks= [checkpoint_callback_2])

Epoch 1/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - loss: 591.8720 - mse: 591.8720 
Epoch 1: saving model to /content/drive/MyDrive/my_srcnn_2_model_epoch_01.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 311s 19s/step - loss: 591.5726 - mse: 591.5726
Epoch 2/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - loss: 581.0358 - mse: 581.0358 
Epoch 2: saving model to /content/drive/MyDrive/my_srcnn_2_model_epoch_02.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 322s 19s/step - loss: 580.6670 - mse: 580.6670
Epoch 3/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - loss: 574.7852 - mse: 574.7852 
Epoch 3: saving model to /content/drive/MyDrive/my_srcnn_2_model_epoch_03.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 323s 19s/step - loss: 574.1121 - mse: 574.1121
Epoch 4/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 19s/step - loss: 567.7859 - mse: 567.7859 
Epoch 4: saving model to /content/drive/MyDrive/my_srcnn_2_model_epoch_04.keras
16/16 ━━━━━━━━━━━━━━━━━━━━ 322s 19s/step - loss: 566.7579 - mse: 566.7579
Epoch 5/10
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 20s/ste

In [ ]:
srcnn_model.save('/content/drive/MyDrive/my_srcnn_model.keras')

In [16]:
srcnn_model = load_model('/content/drive/MyDrive/my_srcnn_2_model_epoch_10.keras')

In [17]:
loss, accuracy = srcnn_model.evaluate(test_images_LR[:10], test_images[:10])
print(f'Test accuracy: {accuracy:.4f}')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - loss: 201.0762 - mse: 201.0762
Test accuracy: 201.0762


#new model without GAN

In [13]:
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import ModelCheckpoint

In [15]:
import tensorflow as tf
from tensorflow.keras import layers, models

def residual_block(x, filters, kernel_size):
    y = layers.Conv2D(filters, kernel_size, padding='same', activation='relu')(x)
    y = layers.Conv2D(filters, kernel_size, padding='same')(y)
    return layers.Add()([x, y])

def build_srcnn_new(f1=9, f2=1, f3=5, n1=64, n2=32, c=3):
    input_img = layers.Input(shape=(256, 256, c))
    x = layers.Conv2D(n1, (f1, f1), padding='same', activation='relu')(input_img)
    x = layers.Conv2D(n2, (f2, f2), padding='same', activation='relu')(x)
    for _ in range(3):
        x = residual_block(x, n2, (f2, f2))
    x = layers.Conv2D(c, (f3, f3), padding='same')(x)
    model = models.Model(input_img, x)
    return model

def compile_srcnn_new(model, learning_rate=1e-4):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mse', metrics=['mse'])

In [16]:
if __name__ == "__main__":

    srcnn_model_new = build_srcnn_new(f1=9, f2=1, f3=5, n1=64, n2=32, c=3)
    compile_srcnn_new(srcnn_model_new, learning_rate=1e-4)
    srcnn_model_new.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 256, 256, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d (Conv2D)           │ (None, 256, 256, 64)   │         15,616 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_1 (Conv2D)         │ (None, 256, 256, 32)   │          2,080 │ conv2d[0][0]           │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_2 (Conv2D)         │ (None, 256, 256, 32)   │          1,056 │ conv2d_1[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_3 (Conv2D)         │ (None, 256, 256, 32)   │          1,056 │ conv2d_2[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add (Add)                 │ (None, 256, 256, 32)   │              0 │ conv2d_1[0][0],        │
│                           │                        │                │ conv2d_3[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_4 (Conv2D)         │ (None, 256, 256, 32)   │          1,056 │ add[0][0]              │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_5 (Conv2D)         │ (None, 256, 256, 32)   │          1,056 │ conv2d_4[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_1 (Add)               │ (None, 256, 256, 32)   │              0 │ add[0][0],             │
│                           │                        │                │ conv2d_5[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_6 (Conv2D)         │ (None, 256, 256, 32)   │          1,056 │ add_1[0][0]            │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_7 (Conv2D)         │ (None, 256, 256, 32)   │          1,056 │ conv2d_6[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ add_2 (Add)               │ (None, 256, 256, 32)   │              0 │ add_1[0][0],           │
│                           │                        │                │ conv2d_7[0][0]         │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ conv2d_8 (Conv2D)         │ (None, 256, 256, 3)    │          2,403 │ add_2[0][0]            │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 26,435 (103.26 KB)

 Trainable params: 26,435 (103.26 KB)

 Non-trainable params: 0 (0.00 B)

In [14]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard

checkpoint_callback_new = ModelCheckpoint(
    filepath='/content/drive/MyDrive/Cv3_models/my_srcnn_new_model_epoch_{epoch:02d}.keras',
    save_best_only=False,
    verbose=1
)


early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,  # Stop after 5 epochs without improvement
    restore_best_weights=True
)

tensorboard = TensorBoard(log_dir='./logs')

In [14]:
srcnn_model_new.fit(train_images_LR[:1000], train_images[:1000], batch_size=BATCH_SIZE, epochs=20, callbacks= [checkpoint_callback_new, early_stopping, tensorboard])

NameError: name 'srcnn_model_new' is not defined

In [15]:
s_model = load_model('/content/drive/MyDrive/Cv3_models/my_srcnn_new_model_epoch_01.keras')

In [ ]:
s_model.fit(train_images_LR[:1000], train_images[:1000], batch_size=BATCH_SIZE, epochs=1, callbacks= [checkpoint_callback_new, early_stopping, tensorboard])

11/16 ━━━━━━━━━━━━━━━━━━━━ 2:01 24s/step - loss: 458.9961 - mse: 458.9961